In [2]:
import pandas as pd
file_path = './refer/output/'

#### age

In [ ]:
# 데이터 로드
file_path = 'processing_survey.csv'
survey_df = pd.read_csv(file_path)

# 나이 범위를 문자열로 변환
survey_df['age_range'] = pd.cut(survey_df['age'], bins=[0, 14, 19, 24, 29, 45], labels=['0-14', '15-19', '20-24', '25-29', '30-45'])

# 나이 범위별 평균 계산
age_mean = survey_df.groupby('age_range').mean().drop(columns=['age', 'job', 'gender','mbti'])

# 결과 출력
print(age_mean)

# 결과를 파일로 저장 (필요 시)
age_mean.to_csv('age_averages.csv')

#### gender

In [ ]:
# 데이터 로드
file_path = 'processing_survey3.csv'
survey_df = pd.read_csv(file_path)

# gender 컬럼을 기준으로 평균 계산
gender_mean = survey_df.groupby('gender').mean()

# 사용하지 않을 컬럼 드롭 (원하는 경우)
columns_to_drop = ['age', 'job', 'mbti']
gender_mean.drop(columns=columns_to_drop, inplace=True)

# 결과 출력
print(gender_mean)

# 결과를 파일로 저장 (필요 시)
gender_mean.to_csv('gender_averages.csv')


#### job

In [ ]:
# 데이터 로드
file_path = 'processing_survey3.csv'
survey_df = pd.read_csv(file_path)

# job 컬럼을 세자리수 문자열로 변환
survey_df['job'] = survey_df['job'].apply(lambda x: f"{int(x):03d}")

# 직업별 평균 계산 (age, job, gender 컬럼 제외)
job_mean = survey_df.groupby('job').mean().drop(columns=['age', 'gender','mbti'])

# 결과 출력
print(job_mean)

# 결과를 파일로 저장 (필요 시)
job_mean.to_csv('job_averages.csv')

#### mbti

In [ ]:
# 데이터 로드
file_path = 'processing_survey3.csv'
survey_df = pd.read_csv(file_path)

# mbti 컬럼을 두자리수 문자열로 변환
survey_df['mbti'] = survey_df['mbti'].apply(lambda x: f"{int(x):02d}")

# MBTI별 평균 계산 (age, job, gender 컬럼 제외)
mbti_mean = survey_df.groupby('mbti').mean().drop(columns=['age','job' ,'gender'])

# 결과 출력
print(mbti_mean)

# 결과를 파일로 저장 (필요 시)
mbti_mean.to_csv('mbti_averages.csv')

-----------------

### base_data와 survey_data 통합
- 논문에서 근거를 얻어 작성한 base_data와 survey_data의 평균을 통합
- 하나의 테이블인 base_weight로 작성

In [32]:
# 모든 업로드된 파일을 로드
file_path = './refer/output'

base_age = pd.read_csv(f'{file_path}/base_age.csv')
base_gender = pd.read_csv(f'{file_path}/base_gender.csv')
base_job = pd.read_csv(f'{file_path}/base_job.csv')
base_mbti = pd.read_csv(f'{file_path}/base_mbti.csv')
survey_gender = pd.read_csv(f'{file_path}/survey_gender.csv')
survey_job = pd.read_csv(f'{file_path}/survey_job.csv')
survey_mbti = pd.read_csv(f'{file_path}/survey_mbti.csv')
survey_age = pd.read_csv(f'{file_path}/survey_age.csv')

In [ ]:
# 데이터프레임을 각 카테고리별로 연결
age = pd.concat([base_age, survey_age]).reset_index(drop=True)
gender = pd.concat([base_gender, survey_gender]).reset_index(drop=True)
job = pd.concat([base_job, survey_job]).reset_index(drop=True)
mbti = pd.concat([base_mbti, survey_mbti]).reset_index(drop=True)

In [ ]:
# 각 데이터프레임을 'age', 'gender', 'job', 'mbti' 열을 기준으로 그룹화하고 각 그룹의 평균을 계산
age_avg = age.groupby('age').mean().reset_index()
gender_avg = gender.groupby('gender').mean().reset_index()
job_avg = job.groupby('job').mean().reset_index()
mbti_avg = mbti.groupby('mbti').mean().reset_index()

In [ ]:
# Cartesian product 병합을 위해 'key' 열 추가
age_avg['key'] = 1
gender_avg['key'] = 1
job_avg['key'] = 1
mbti_avg['key'] = 1

In [ ]:
# 'key' 열을 기준으로 병합 수행
merged = age_avg.merge(gender_avg, on='key', suffixes=('', '_gender'))
merged = merged.merge(job_avg, on='key', suffixes=('', '_job'))
merged = merged.merge(mbti_avg, on='key', suffixes=('', '_mbti'))

# 병합 후 'key' 열 제거
merged = merged.drop(columns=['key'])

In [ ]:
# 'work', 'edu', 'free_time', 'health', 'chores', 'category_else' 열의 값을 합산하여 새로운 열 생성
columns_to_sum = ['work', 'edu', 'free_time', 'health', 'chores', 'category_else']
for col in columns_to_sum:
    merged[col] = merged[[f'{col}', f'{col}_gender', f'{col}_job', f'{col}_mbti']].sum(axis=1)

# 최종 결과 데이터프레임 생성 (필요한 열들만 선택)
final_result = merged[['age', 'gender', 'job', 'mbti', 'work', 'edu', 'free_time', 'health', 'chores', 'category_else']]

In [ ]:
# 결과를 CSV 파일로 저장
final_result.to_csv(f'{file_path}/base_weight.csv', index=False)